Load jupyter sql extension


In [5]:
#Load Jupter SQL extension
%load_ext sql
%config SqlMagic.autocommit = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Next load the database

In [6]:
#Load database
%sql sqlite:///../data/raw/frantamarit.db

'Connected: @../data/raw/frantamarit.db'

Loading the database tables

In [7]:
#Load database tables
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable tables


In [8]:
#Showing tables
tables

name
rank_info
personal_info
business_info


1. Creating engine to connect to the database

In [20]:
#from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

2. Create engine and get data tables

In [24]:
#Creating engine
eng = create_engine('sqlite:///../data/raw/frantamarit.db')

3. Creating data frames for each table

In [26]:
#Creating function to ingest dataframe
def create_df_sql(x):
    return pd.read_sql_table(x, con=eng)

#Defining datafranes
df_rank_info = create_df_sql('business_info')
df_personal_info = create_df_sql('personal_info')
df_rank_info = create_df_sql('rank_info')

In [32]:
len(df_rank_info)
#df_rank_info['Unnamed: 0'].le

2260

In [ ]:
print(rank_info.notnull().sum())
print(personal_info.notnull().sum())

In [ ]:
#Formula to drop column inplace
def drop_col_in(x,y):
    return x.drop(y, axis=1, inplace=True)

drop_col_in()

"""
rank_info.drop('Unnamed: 0', axis=1, inplace=True)
personal_info.drop('Unnamed: 0', axis=1, inplace=True)
business_info.drop('Unnamed: 0', axis=1, inplace=True)
"""

In [ ]:
personal_info.head()

In [ ]:
partial_info = pd.merge(rank_info, personal_info, on='id', how='inner')
master_info = pd.merge(partial_info, business_info, on='id', how='inner')
master_info.head()


In [ ]:
#Cleaning and lowercasing name and lastName features
master_info[['name','lastName']] = \
master_info[['name','lastName']].apply(lambda x: x.astype(str).str.lower())

In [ ]:
master_info.isnull().sum()

In [ ]:
#Renaming feature names
master_info.rename(index={e: e+1 for e in range(len(master_info))}, 
                columns={'name': 'full_name', 
                         'lastName': 'last_name',
                        'Source': 'source'}, 
                inplace=True)

In [ ]:
master_info.head()